In [10]:
import pandas as pd
import preprocessor_improved as prep_improved
import preprocessor_baseline as prep_baseline
from random_forest_regressor import random_forest_regressor
from linear_regression import linear_regression
from gbm import xg_boost
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
warnings.filterwarnings('ignore')



# Load the datasets
singapore_df = pd.read_csv('datasets/singapore_listings.csv')
ny_df = pd.read_csv('datasets/newyorkcity_listings.csv')
madrid_df = pd.read_csv('datasets/madrid_listings.csv')

In [1]:
import pandas as pd
import preprocessor_improved as prep_improved
import preprocessor_baseline as prep_baseline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import warnings
import numpy as np

# Define a function to preprocess the data
def preprocess_data(df):
    preprocessor, X, y = prep_baseline.preprocess_data(df)
    X_transformed = preprocessor.fit_transform(X)
    return preprocessor, X_transformed, y

# Use a subset of the data to speed up testing
subset_fraction = 0.1 # Example fraction, adjust as needed

# Get subsets for each dataset
singapore_df_subset = singapore_df.sample(frac=subset_fraction, random_state=42)
ny_df_subset = ny_df.sample(frac=subset_fraction, random_state=42)
madrid_df_subset = madrid_df.sample(frac=subset_fraction, random_state=42)

# Preprocess the data for each subset
singapore_preprocessor, singapore_X_transformed, singapore_y = preprocess_data(singapore_df)
ny_preprocessor, ny_X_transformed, ny_y = preprocess_data(ny_df)
madrid_preprocessor, madrid_X_transformed, madrid_y = preprocess_data(madrid_df)

# Split the data into training and testing sets
singapore_X_train, singapore_X_test, singapore_y_train, singapore_y_test = train_test_split(
    singapore_X_transformed, singapore_y, test_size=0.3, random_state=42
)
ny_X_train, ny_X_test, ny_y_train, ny_y_test = train_test_split(
    ny_X_transformed, ny_y, test_size=0.3, random_state=42
)
madrid_X_train, madrid_X_test, madrid_y_train, madrid_y_test = train_test_split(
    madrid_X_transformed, madrid_y, test_size=0.3, random_state=42
)

# Define models
rf_model = RandomForestRegressor()
lin_model = LinearRegression()
xgb_model = XGBRegressor()

# Define hyperparameters for each model
rf_params = {
    'n_estimators': [int(x) for x in np.linspace(100, 1000, num=20)],
    'max_features': ['log2', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(10, 150, num=11)],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 10],
    'bootstrap': [True, False],
}

lin_params = {
    'fit_intercept': [True, False],
}

xgb_params = {
    # 'n_estimators': [100, 200],
    # 'learning_rate': [0.01, 0.1, 0.2, 0.3],
    # 'max_depth': [3, 4, 5],
    # 'min_child_weight': [1, 2, 3],
    # 'gamma': [0, 0.1, 0.2],
    # 'subsample': [0.8, 0.9, 1.0],
    # 'colsample_bytree': [0.8, 0.9, 1.0],

    'learning_rate': [0.05, 0.1, 0.15, 0.2],
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.1, 0.2, 0.3, 0.5],
    'reg_lambda': [0, 0.1, 0.2, 0.3, 0.5],
    'gamma': [0, 1, 2],
}

# Perform hyperparameter search with the subsets
def run_hyperparameter_search(model, params, X_train, y_train):
    rnd_search_cv = RandomizedSearchCV(
        model, params, cv=3, n_iter=10, random_state=42, n_jobs=1, scoring='neg_mean_squared_error'
    )
    rnd_search_cv.fit(X_train, y_train)
    return rnd_search_cv


C:\Users\Devon Murray\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [12]:

singapore_rf_search = run_hyperparameter_search(rf_model, rf_params, singapore_X_train, singapore_y_train)
ny_rf_search = run_hyperparameter_search(rf_model, rf_params, ny_X_train, ny_y_train)
madrid_rf_search = run_hyperparameter_search(rf_model, rf_params, madrid_X_train, madrid_y_train)

In [15]:

singapore_lin_search = run_hyperparameter_search(lin_model, lin_params, singapore_X_train, singapore_y_train)
ny_lin_search = run_hyperparameter_search(lin_model, lin_params, ny_X_train, ny_y_train)
madrid_lin_search = run_hyperparameter_search(lin_model, lin_params, madrid_X_train, madrid_y_train)

In [25]:
singapore_xgb_search = run_hyperparameter_search(xgb_model, xgb_params, singapore_X_train, singapore_y_train)
ny_xgb_search = run_hyperparameter_search(xgb_model, xgb_params, ny_X_train, ny_y_train)
madrid_xgb_search = run_hyperparameter_search(xgb_model, xgb_params, madrid_X_train, madrid_y_train)


In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Function to calculate evaluation metrics, including RMSE
def calculate_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calculate RMSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, rmse, mae, r2


In [18]:

# Calculate metrics for Random Forest
singapore_rf_best_model = singapore_rf_search.best_estimator_
ny_rf_best_model = ny_rf_search.best_estimator_
madrid_rf_best_model = madrid_rf_search.best_estimator_

singapore_rf_metrics = calculate_metrics(singapore_rf_best_model, singapore_X_test, singapore_y_test)
ny_rf_metrics = calculate_metrics(ny_rf_best_model, ny_X_test, ny_y_test)
madrid_rf_metrics = calculate_metrics(madrid_rf_best_model, madrid_X_test, madrid_y_test)


print("Singapore Random Forest Metrics (MSE, RMSE, MAE, R^2):", singapore_rf_metrics)
print("New York Random Forest Metrics (MSE, RMSE, MAE, R^2):", ny_rf_metrics)
print("Madrid Random Forest Metrics (MSE, RMSE, MAE, R^2):", madrid_rf_metrics)

Singapore Random Forest Metrics (MSE, RMSE, MAE, R^2): (57011.1682682344, 238.7701159446768, 73.18845001226889, 0.11188169106753854)
New York Random Forest Metrics (MSE, RMSE, MAE, R^2): (31088.147106733395, 176.31831188714744, 61.02936709367881, 0.23024957924696887)
Madrid Random Forest Metrics (MSE, RMSE, MAE, R^2): (250884.37523552735, 500.8835944962935, 109.0362626233096, 0.06255570262047006)


In [19]:
# Calculate metrics for Linear Regression
singapore_lin_best_model = singapore_lin_search.best_estimator_
ny_lin_best_model = ny_lin_search.best_estimator_
madrid_lin_best_model = madrid_lin_search.best_estimator_

singapore_lin_metrics = calculate_metrics(singapore_lin_best_model, singapore_X_test, singapore_y_test)
ny_lin_metrics = calculate_metrics(ny_lin_best_model, ny_X_test, ny_y_test)
madrid_lin_metrics = calculate_metrics(madrid_lin_best_model, madrid_X_test, madrid_y_test)


print("Singapore Linear Regression Metrics (MSE, RMSE, MAE, R^2):", singapore_lin_metrics)
print("New York Linear Regression Metrics (MSE, RMSE, MAE, R^2):", ny_lin_metrics)
print("Madrid Linear Regression Metrics (MSE, RMSE, MAE, R^2):", madrid_lin_metrics)

Singapore Linear Regression Metrics (MSE, RMSE, MAE, R^2): (61786.764526406245, 248.56943602624648, 84.68060486827693, 0.037487592476246556)
New York Linear Regression Metrics (MSE, RMSE, MAE, R^2): (34298.93057301481, 185.1997045705387, 69.23455437503382, 0.15074976487618263)
Madrid Linear Regression Metrics (MSE, RMSE, MAE, R^2): (264559.7972898442, 514.3537666721652, 124.29027250913583, 0.01145667978557896)


In [26]:
# Calculate metrics for XGBoost Regressor
singapore_xgb_best_model = singapore_xgb_search.best_estimator_
ny_xgb_best_model = ny_xgb_search.best_estimator_
madrid_xgb_best_model = madrid_xgb_search.best_estimator_

singapore_xgb_metrics = calculate_metrics(singapore_xgb_best_model, singapore_X_test, singapore_y_test)
ny_xgb_metrics = calculate_metrics(ny_xgb_best_model, ny_X_test, ny_y_test)
madrid_xgb_metrics = calculate_metrics(madrid_xgb_best_model, madrid_X_test, madrid_y_test)

print("Singapore XGBoost Metrics (MSE, RMSE, MAE, R^2):", singapore_xgb_metrics)
print("New York XGBoost Metrics (MSE, RMSE, MAE, R^2):", ny_xgb_metrics)
print("Madrid XGBoost Metrics (MSE, RMSE, MAE, R^2):", madrid_xgb_metrics)

Singapore XGBoost Metrics (MSE, RMSE, MAE, R^2): (86558.06894897319, 294.2075270093768, 81.59877991204115, -0.34839906205279725)
New York XGBoost Metrics (MSE, RMSE, MAE, R^2): (33823.84921415189, 183.91261298277476, 63.44118624787055, 0.16251290002287178)
Madrid XGBoost Metrics (MSE, RMSE, MAE, R^2): (248128.63263587176, 498.12511745130035, 117.16333132624061, 0.07285269773093772)


In [2]:

# Preprocess the data
singapore_preproccessor, singapore_X, singapore_y = prep_baseline.preprocess_data(singapore_df)
singapore_X_transformed = singapore_preproccessor.fit_transform(singapore_X)
ny_preproccessor, ny_X, ny_y = prep_baseline.preprocess_data(ny_df)
madrid_preproccessor, madrid_X, madrid_y = prep_baseline.preprocess_data(madrid_df)


# Split the data into training and testing sets
singapore_X_train, singapore_X_test, singapore_y_train, singapore_y_test = train_test_split(singapore_X_transformed, singapore_y, test_size=0.3, random_state=42)

rf_model = RandomForestRegressor()
lin_model=LinearRegressor()
gbm_model=GradientBoostingRegressor()

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 20)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 150, num = 11)]
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 10, 20]
bootstrap = [True, False]

parametrs = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rnd_search_cv = RandomizedSearchCV(model, parametrs, cv=5)
rnd_search_cv.fit(singapore_X_train, singapore_y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 24, 38, 52, 66, 80,
                                                      94, 108, 122, 136, 150],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 10, 20],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [100, 147, 194, 242,
                                                         289, 336, 384, 431,
                                                         478, 526, 573, 621,
                                                         668, 715, 763, 810,
                                                         857, 905, 952, 1000]})

In [4]:
from sklearn.ensemble import GradientBoostingRegressor

# Preprocess the data
singapore_preproccessor, singapore_X, singapore_y = prep_baseline.preprocess_data(singapore_df)
singapore_X_transformed = singapore_preproccessor.fit_transform(singapore_X)

# Split the data into training and testing sets
singapore_X_train, singapore_X_test, singapore_y_train, singapore_y_test = train_test_split(singapore_X_transformed, singapore_y, test_size=0.3, random_state=42)

# Define the GBM model
model = GradientBoostingRegressor()

# Define hyperparameters for tuning
parametrs = {
    'n_estimators': [100, 200, 300],  # Number of boosting stages to be run
    'learning_rate': [0.05, 0.1, 0.2],  # Boosting learning rate
    'max_depth': [3, 4, 5],  # Maximum depth of the individual estimators
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'subsample': [0.8, 0.9, 1.0]  # Fraction of samples used for fitting the individual base learners
}

# Perform Randomized Search Cross Validation
rnd_search_cv = RandomizedSearchCV(model, parametrs, cv=5)
rnd_search_cv.fit(singapore_X_train, singapore_y_train)


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(),
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.8, 0.9, 1.0]})

In [3]:
predictions = rnd_search_cv.predict(singapore_X_test)

#Mean Absolute Error 
print('MAE:', metrics.mean_absolute_error(singapore_y_test, predictions))

#Mean Squared Error
print('MSE:', metrics.mean_squared_error(singapore_y_test, predictions))


#Root Mean Squared Error
print('RMSE:', np.sqrt(metrics.mean_squared_error(singapore_y_test, predictions)))

#R2
print('R2:', metrics.r2_score(singapore_y_test, predictions))

MAE: 75.56751131152274
MSE: 57299.51008290595
RMSE: 239.3731607405182
R2: 0.1073899107266103


In [5]:
predictions = rnd_search_cv.predict(singapore_X_test)
print('Gradient Boosting Regressor Results')
#Mean Absolute Error 
print('MAE for gr:', metrics.mean_absolute_error(singapore_y_test, predictions))

#Mean Squared Error
print('MSE:', metrics.mean_squared_error(singapore_y_test, predictions))


#Root Mean Squared Error
print('RMSE:', np.sqrt(metrics.mean_squared_error(singapore_y_test, predictions)))

#R2
print('R2:', metrics.r2_score(singapore_y_test, predictions))

Gradient Boosting Regressor Results
MAE for gr: 86.95818144897702
MSE: 74203.2711027741
RMSE: 272.4027736693848
R2: -0.15593638318356873


In [6]:
pd.Series(rnd_search_cv.best_estimator_.feature_importances_, index=singapore_df.columns).sort_values(ascending=False)

ValueError: Length of values (58) does not match length of index (20)